In [1]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from model import ConsumptionSavingModel
from algos import TISolver, EGMSolver, ForwardRolloutSolver
from plot import plot_policy_comparison, plot_computation_time, plot_convergence_paths
import time

# Set random seed for reproducibility
np.random.seed(42)

In [2]:
# Model parameters (typical values from literature)
beta = 0.96  # discount factor
R = 1.04     # gross interest rate (4% annual)
sigma = 2.0  # relative risk aversion
b = 0.0      # borrowing constraint (natural borrowing limit)

# Grid parameters
M = 100  # number of capital grid points
N = 5    # number of wage states

# Create capital grid
k_min = b
k_max = 20.0  # upper bound for capital
k_grid = np.linspace(k_min, k_max, M)  # linear grid for simplicity

In [3]:
# Set up wage process (log-AR(1))
rho = 0.9       # persistence
sigma_w = 0.2   # wage shock volatility
w_mean = 1.0    # mean wage

# Create wage grid using Tauchen method
w_grid = np.exp(np.linspace(-2*sigma_w, 2*sigma_w, N))
w_grid = w_grid * w_mean / np.mean(w_grid)  # normalize mean

# Create transition matrix
P = np.zeros((N, N))
z_grid = np.log(w_grid)
z_step = (z_grid[-1] - z_grid[0]) / (N - 1)

for i in range(N):
    for j in range(N):
        if j == 0:
            P[i, j] = norm.cdf((z_grid[0] - rho * z_grid[i] + z_step/2) / sigma_w)
        elif j == N-1:
            P[i, j] = 1 - norm.cdf((z_grid[-1] - rho * z_grid[i] - z_step/2) / sigma_w)
        else:
            P[i, j] = (norm.cdf((z_grid[j] - rho * z_grid[i] + z_step/2) / sigma_w) - 
                      norm.cdf((z_grid[j] - rho * z_grid[i] - z_step/2) / sigma_w))

# Verify row sums are close to 1
print("Transition matrix row sums:", P.sum(axis=1))

Transition matrix row sums: [1. 1. 1. 1. 1.]


In [4]:
# Create model instance and solvers
model = ConsumptionSavingModel(beta, R, sigma, b, k_grid, w_grid, P)

# Initialize solvers with different settings
ti_solver = TISolver(model, max_iter=1000, tol=1e-6)
egm_solver = EGMSolver(model, max_iter=1000, tol=1e-6)
fr_solver = ForwardRolloutSolver(model, max_iter=1000, tol=1e-6)

# Dictionary to store computation times and policies
computation_times = {}
policies = {}

In [ ]:
# Solve with Time Iteration
print("Solving with Time Iteration...")
start_time = time.time()
c_ti, k_ti = ti_solver.solve(damping=0.5)
computation_times['Time Iteration'] = time.time() - start_time
policies['Time Iteration'] = (c_ti, k_ti)
print(f"Time Iteration completed in {computation_times['Time Iteration']:.2f} seconds\n")

# Solve with EGM
print("Solving with EGM...")
start_time = time.time()
c_egm, k_egm = egm_solver.solve(damping=1.0)
computation_times['EGM'] = time.time() - start_time
policies['EGM'] = (c_egm, k_egm)
print(f"EGM completed in {computation_times['EGM']:.2f} seconds\n")

# Solve with Forward Rollout
print("Solving with Forward Rollout...")
start_time = time.time()
c_fr, k_fr = fr_solver.solve(damping=0.3)
computation_times['Forward Rollout'] = time.time() - start_time
policies['Forward Rollout'] = (c_fr, k_fr)
print(f"Forward Rollout completed in {computation_times['Forward Rollout']:.2f} seconds")

Solving with Time Iteration...


## Results Visualization and Analysis

Let's compare the solutions across different methods by looking at:
1. Policy functions for the middle wage state
2. Computation times
3. Maximum differences between solutions
4. Policy statistics

In [ ]:
# Plot policy functions for middle wage state
middle_w_idx = N // 2
fig_policy = plot_policy_comparison(model, policies, w_idx=middle_w_idx)
plt.show()

# Plot computation times
fig_times = plot_computation_time(computation_times)
plt.show()

# Calculate maximum differences between methods
print("\nMaximum policy differences:")
print(f"TI vs EGM: {np.max(np.abs(c_ti - c_egm)):.2e}")
print(f"TI vs FR: {np.max(np.abs(c_ti - c_fr)):.2e}")
print(f"EGM vs FR: {np.max(np.abs(c_egm - c_fr)):.2e}")

# Print final policy statistics
print("\nPolicy statistics (for middle wage state):")
for method_name, (c, k) in policies.items():
    print(f"\n{method_name}:")
    print(f"Mean consumption: {np.mean(c[:, middle_w_idx]):.3f}")
    print(f"Mean savings rate: {np.mean((model.R * k_grid + w_grid[middle_w_idx] - c[:, middle_w_idx]) / (model.R * k_grid + w_grid[middle_w_idx])):.3f}")
    print(f"Min consumption: {np.min(c[:, middle_w_idx]):.3f}")
    print(f"Max consumption: {np.max(c[:, middle_w_idx]):.3f}")

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from model import ConsumptionSavingModel
from algos import TISolver, EGMSolver, ForwardRolloutSolver
from plot import plot_policy_comparison, plot_computation_time, plot_convergence_paths
import time

# Set random seed for reproducibility
np.random.seed(42)

# Model parameters (typical values from literature)
beta = 0.96  # discount factor
R = 1.04     # gross interest rate (4% annual)
sigma = 2.0  # relative risk aversion
b = 0.0      # borrowing constraint (natural borrowing limit)

# Grid parameters
M = 100  # number of capital grid points
N = 5    # number of wage states

# Create grids
k_min = b
k_max = 20.0  # upper bound for capital
k_grid = np.linspace(k_min, k_max, M)  # linear grid for simplicity
# Alternative: exponential grid for better resolution at low values
# k_grid = k_min + (k_max - k_min) * (np.exp(np.linspace(0, 1, M)) - 1) / (np.exp(1) - 1)

# Wage grid and transition matrix (log-AR(1) process for wages)
rho = 0.9  # persistence
sigma_w = 0.2  # wage shock volatility
w_mean = 1.0  # mean wage

# Create wage grid using Tauchen method
w_grid = np.exp(np.linspace(-2*sigma_w, 2*sigma_w, N))
w_grid = w_grid * w_mean / np.mean(w_grid)  # normalize mean

# Create transition matrix
P = np.zeros((N, N))
z_grid = np.log(w_grid)
z_step = (z_grid[-1] - z_grid[0]) / (N - 1)

for i in range(N):
    for j in range(N):
        if j == 0:
            P[i, j] = norm.cdf((z_grid[0] - rho * z_grid[i] + z_step/2) / sigma_w)
        elif j == N-1:
            P[i, j] = 1 - norm.cdf((z_grid[-1] - rho * z_grid[i] - z_step/2) / sigma_w)
        else:
            P[i, j] = (norm.cdf((z_grid[j] - rho * z_grid[i] + z_step/2) / sigma_w) - 
                      norm.cdf((z_grid[j] - rho * z_grid[i] - z_step/2) / sigma_w))

# Create model instance
model = ConsumptionSavingModel(beta, R, sigma, b, k_grid, w_grid, P)

# Initialize solvers
ti_solver = TISolver(model, max_iter=1000, tol=1e-6)
egm_solver = EGMSolver(model, max_iter=1000, tol=1e-6)
fr_solver = ForwardRolloutSolver(model, max_iter=1000, tol=1e-6)

# Solve with each method and time it
computation_times = {}
policies = {}

# Time Iteration
start_time = time.time()
c_ti, k_ti = ti_solver.solve(damping=0.5)
computation_times['Time Iteration'] = time.time() - start_time
policies['Time Iteration'] = (c_ti, k_ti)

# EGM
start_time = time.time()
c_egm, k_egm = egm_solver.solve(damping=1.0)
computation_times['EGM'] = time.time() - start_time
policies['EGM'] = (c_egm, k_egm)

# Forward Rollout
start_time = time.time()
c_fr, k_fr = fr_solver.solve(damping=0.3)
computation_times['Forward Rollout'] = time.time() - start_time
policies['Forward Rollout'] = (c_fr, k_fr)

# Plot policy functions for middle wage state
middle_w_idx = N // 2
fig_policy = plot_policy_comparison(model, policies, w_idx=middle_w_idx)
plt.show()

# Plot computation times
fig_times = plot_computation_time(computation_times)
plt.show()

# Calculate maximum differences between methods
print("\nMaximum policy differences:")
print(f"TI vs EGM: {np.max(np.abs(c_ti - c_egm)):.2e}")
print(f"TI vs FR: {np.max(np.abs(c_ti - c_fr)):.2e}")
print(f"EGM vs FR: {np.max(np.abs(c_egm - c_fr)):.2e}")

# Print final policy statistics
print("\nPolicy statistics (for middle wage state):")
for method_name, (c, k) in policies.items():
    print(f"\n{method_name}:")
    print(f"Mean consumption: {np.mean(c[:, middle_w_idx]):.3f}")
    print(f"Mean savings rate: {np.mean((model.R * k_grid + w_grid[middle_w_idx] - c[:, middle_w_idx]) / (model.R * k_grid + w_grid[middle_w_idx])):.3f}")
    print(f"Min consumption: {np.min(c[:, middle_w_idx]):.3f}")
    print(f"Max consumption: {np.max(c[:, middle_w_idx]):.3f}")

NameError: name 'norm' is not defined